In [1]:
import torch
torch.cuda.empty_cache()
# Imports
from transformers import AutoTokenizer, TrainingArguments, Trainer, AutoModelForSequenceClassification,TrainerCallback
import torch
from transformers import AutoModelForMaskedLM
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
import numpy as np
from datasets import load_dataset, Dataset
import scipy.special
import os
import json
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# Define the working device
import csv
import pandas as pd
from Bio import SeqIO
from Bio.SeqUtils.ProtParam import ProteinAnalysis

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
checkpoint_path='/home/huang/chy/ESMC/esm plus'
model = AutoModelForSequenceClassification.from_pretrained(checkpoint_path,trust_remote_code=True).to(device).eval()
tokenizer = model.tokenizer
vocab = tokenizer.get_vocab()


Some weights of ESMplusplusForSequenceClassification were not initialized from the model checkpoint at /home/huang/chy/ESMC/esm plus and are newly initialized: ['classifier.0.bias', 'classifier.0.weight', 'classifier.2.bias', 'classifier.2.weight', 'classifier.3.bias', 'classifier.3.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
##################################################
#############RHCH38序列

TP53_pro=SeqIO.read(r"/home/huang/chy/数据处理/p53_protein_sequence.fasta", "fasta")
print(len(TP53_pro))
print(type(TP53_pro))
TP53_pro_seq=TP53_pro.seq
print(type(TP53_pro_seq))
WT_seq=str(TP53_pro_seq)

393
<class 'Bio.SeqRecord.SeqRecord'>
<class 'Bio.Seq.Seq'>


In [4]:
import torch
import torch.nn.functional as F
from tqdm import tqdm
import pandas as pd

esm_distance_list = []
WT_avg_list=[]
MUT_avg_list=[]
# 遍历DataFrame
for index, row in tqdm(df_gn.iterrows(), total=df_gn.shape[0]):
    WT_seq = row['wt_seq']
    MUT_seq = row['mut_seq']
    
    # 对WT序列进行编码
    WT_seq_tokenized = tokenizer(WT_seq, padding=True, return_tensors='pt')
    WT_seq_tokenized.to(device)
    
    # 获取WT序列的last_hidden_state
    output = model(**WT_seq_tokenized)
    WT_last_hidden_state = output.last_hidden_state
    
    # 对MUT序列进行编码
    MUT_seq_tokenized = tokenizer(MUT_seq, padding=True, return_tensors='pt')
    MUT_seq_tokenized.to(device)
    
    # 获取MUT序列的last_hidden_state
    MUT_output = model(**MUT_seq_tokenized)
    MUT_last_hidden_state = MUT_output.last_hidden_state
    
    # 计算整体嵌入向量
    WT_avg_hidden_state = torch.mean(WT_last_hidden_state, dim=1)
    MUT_avg_hidden_state = torch.mean(MUT_last_hidden_state, dim=1)
    
    # 计算欧氏距离
    with torch.no_grad():
        distance = torch.norm(WT_avg_hidden_state - MUT_avg_hidden_state)
    WT_avg_hidden_state = WT_avg_hidden_state.squeeze().detach().cpu().numpy()
    MUT_avg_hidden_state = MUT_avg_hidden_state.squeeze().detach().cpu().numpy()
    # 将距离转换为CPU上的NumPy数组，并提取其值
    distance = distance.cpu().numpy().item()
    row['esm_distance_base'] = distance
    #row['WT_rep'] = WT_last_hidden_state.squeeze().detach().cpu().numpy()
    #row['MUT_rep'] = MUT_last_hidden_state.squeeze().detach().cpu().numpy()
    esm_distance_list.append(distance)
    WT_avg_list.append(WT_last_hidden_state.squeeze().detach().cpu().numpy())
    MUT_avg_list.append(MUT_last_hidden_state.squeeze().detach().cpu().numpy())

    del WT_seq_tokenized,MUT_seq_tokenized,output,MUT_output
    torch.cuda.empty_cache()
#df_gn['WT_rep']=WT_avg_list
#df_gn['MUT_rep']=MUT_avg_list
df_gn['esm_distance_pretain']=esm_distance_list
print(df_gn.head())



100%|██████████| 7467/7467 [07:44<00:00, 16.07it/s]

  Name REF  pos ALT  labels  \
0  M1K   M    1   K       2   
1  M1R   M    1   R       2   
2  M1H   M    1   H       2   
3  M1E   M    1   E       2   
4  M1D   M    1   D       2   

                                              wt_seq  \
0  <pad><pad><pad><pad><pad><pad><pad><pad><pad><...   
1  <pad><pad><pad><pad><pad><pad><pad><pad><pad><...   
2  <pad><pad><pad><pad><pad><pad><pad><pad><pad><...   
3  <pad><pad><pad><pad><pad><pad><pad><pad><pad><...   
4  <pad><pad><pad><pad><pad><pad><pad><pad><pad><...   

                                             mut_seq  esm_distance_base  \
0  <pad><pad><pad><pad><pad><pad><pad><pad><pad><...           0.118096   
1  <pad><pad><pad><pad><pad><pad><pad><pad><pad><...           0.081521   
2  <pad><pad><pad><pad><pad><pad><pad><pad><pad><...           0.073199   
3  <pad><pad><pad><pad><pad><pad><pad><pad><pad><...           0.125566   
4  <pad><pad><pad><pad><pad><pad><pad><pad><pad><...           0.107556   

   esm_distance_pretain  

In [1]:
import torch
torch.cuda.empty_cache()
# Imports
from transformers import AutoTokenizer, TrainingArguments, Trainer, AutoModelForSequenceClassification,TrainerCallback
import torch
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
import numpy as np
from datasets import load_dataset, Dataset
import scipy.special
import os
import json
import torch.nn.functional as F
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# Define the working device
import csv
import pandas as pd
from tqdm import tqdm 

def run_inference(checkpoint_path, input_path, output_path, output_format):
    # Define device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load model
    model = AutoModelForSequenceClassification.from_pretrained(checkpoint_path,trust_remote_code=True).to(device)

    # Load tokenizer
    tokenizer =  model.tokenizer

    # Load data
    data=pd.read_csv(input_path)


    # Run model
    results = []
    for index, row in tqdm(data.iterrows(), total=len(data), desc="Processing"):
        seq=row['mut_seq']
        inputs = tokenizer(seq, return_tensors="pt", padding=True, truncation=True, max_length=601).to(device)
        outputs = model(**inputs)
        logits = outputs.logits
        probabilities = F.softmax(logits, dim=-1)
        predicted_class = logits.argmax(dim=-1).item()
        confidence = probabilities[0, predicted_class].item()
        y_score = probabilities[0, 1].item()  # 获取正类（类别1）的概率
        new_row=row.copy()
        new_row['esmc_pre_label']=predicted_class
        new_row['confidence_score']=confidence
        new_row['y_score'] = y_score  # 添加 y_scores 列
        results.append(new_row)

    results_df = pd.DataFrame(results)
    # Save results
    if output_format == "json":
        with open(output_path, 'w') as outfile:
            json.dump(results_df.to_dict(orient='records'), outfile, indent=4)
    elif output_format == "csv":
        with open(output_path, mode='w', newline='') as file:
            results_df.to_csv(output_path, index=False)
    else:
        raise ValueError("Unsupported output format. Use 'json' or 'csv'.")

In [ ]:
import torch
import pandas as pd
from tqdm import tqdm
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import numpy as np
import scipy.spatial.distance as sp

def run_inference(checkpoint_path, input_path, output_path, output_format):
    # Define device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load model
    model = AutoModelForSequenceClassification.from_pretrained(checkpoint_path, trust_remote_code=True).to(device)

    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)

    # Load data
    data = pd.read_csv(input_path)

    # Run model
    results = []
    esm_distance_list = []
    WT_avg_list = []
    MUT_avg_list = []

    for index, row in tqdm(data.iterrows(), total=len(data), desc="Processing"):
        seq = row['mut_seq']
        inputs = tokenizer(seq, return_tensors="pt", padding=True, truncation=True, max_length=601).to(device)
        outputs = model(**inputs)
        logits = outputs.logits
        probabilities = torch.nn.functional.softmax(logits, dim=-1)
        predicted_class = logits.argmax(dim=-1).item()
        confidence = probabilities[0, predicted_class].item()
        y_score = probabilities[0, 1].item()  # 获取正类（类别1）的概率
        new_row = row.copy()
        new_row['esmc_pre_label'] = predicted_class
        new_row['confidence_score'] = confidence
        new_row['y_score'] = y_score  # 添加 y_scores 列
        results.append(new_row)

        WT_seq = row['wt_seq']
        MUT_seq = row['mut_seq']

        # 对序列进行编码
        WT_seq_tokenized = tokenizer(WT_seq, padding=True, return_tensors='pt')
        WT_seq_tokenized.to(device)
        
        # 获取WT序列的last_hidden_state
        output = model(**WT_seq_tokenized)
        WT_last_hidden_state = output.last_hidden_state
        
        # 对MUT序列进行编码
        MUT_seq_tokenized = tokenizer(MUT_seq, padding=True, return_tensors='pt')
        MUT_seq_tokenized.to(device)
        
        # 获取MUT序列的last_hidden_state
        MUT_output = model(**MUT_seq_tokenized)
        MUT_last_hidden_state = MUT_output.last_hidden_state
        
        # 计算整体嵌入向量
        WT_avg_hidden_state = torch.mean(WT_last_hidden_state, dim=1)
        MUT_avg_hidden_state = torch.mean(MUT_last_hidden_state, dim=1)
        
        # 计算欧氏距离
        with torch.no_grad():
            distance = torch.norm(WT_avg_hidden_state - MUT_avg_hidden_state)
        WT_avg_hidden_state = WT_avg_hidden_state.squeeze().detach().cpu().numpy()
        MUT_avg_hidden_state = MUT_avg_hidden_state.squeeze().detach().cpu().numpy()
        # 将距离转换为CPU上的NumPy数组，并提取其值
        distance = distance.cpu().numpy().item()
        row['esm_distance_base'] = distance
        row['WT_rep'] = WT_avg_hidden_state
        #row['MUT_rep'] = MUT_last_hidden_state
        esm_distance_list.append(distance)
        WT_avg_list.append(WT_last_hidden_state.squeeze().detach().cpu().numpy())
        MUT_avg_list.append(MUT_last_hidden_state.squeeze().detach().cpu().numpy())

        del WT_seq_tokenized,MUT_seq_tokenized,output,MUT_output
        torch.cuda.empty_cache()

    results_df = pd.DataFrame(results)
    # Save results
    if output_format == "json":
        with open(output_path, 'w') as outfile:
            json.dump(results_df.to_dict(orient='records'), outfile, indent=4)
    elif output_format == "csv":
        with open(output_path, mode='w', newline='') as file:
            results_df.to_csv(output_path, index=False)
    else:
        raise ValueError("Unsupported output format. Use 'json' or 'csv'.")


In [2]:
checkpoint_path='chenyuhe/CaVepP53'
input_path='/home/huang/chy/P53预测最终/specific-model/最终评估结果总汇/test&val_data_finetune_compare.csv'
output_path='/home/huang/chy/P53预测最终/specific-model/最终评估结果总汇/mode_test.csv'
output_format="csv"
run_inference(checkpoint_path, input_path, output_path, output_format)

Processing: 100%|██████████| 705/705 [00:09<00:00, 72.37it/s]


In [1]:
from score import run_inference
checkpoint_path='chenyuhe/CaVepP53'
input_path='./data/example_pre_data.csv'
output_path='./data/example_outcome.csv'
run_inference(checkpoint_path, input_path, output_path)

Predicting: 100%|██████████| 705/705 [00:31<00:00, 22.53it/s]
